In [1]:
import pandas as pd
import numpy as np

In [2]:
# import master vocaulary with bigram classifications
bigrams_master = bigrams = pd.read_csv("gs://rwc1/data/vocabulary/master_list.txt", sep = "|")


In [3]:
# improt speaker bigram for session X
speaker_bigrams = pd.read_csv("gs://rwc1/data/hein-bound/byspeaker_2gram_111.txt", sep = "|")

# import speaker map for session X
speaker_map = pd.read_csv("gs://rwc1/data/hein-bound/111_SpeakerMap.txt", sep = "|")

In [4]:
print(bigrams_master.shape)
bigrams_master.head()

(6822118, 2)


,phrase,_classify
0,0 0,bad_syntax
1,0 00,bad_syntax
2,0 000,bad_syntax
3,0 0000,bad_syntax
4,0 00000,bad_syntax


In [5]:
print(speaker_bigrams.shape)
speaker_bigrams.head()

(10467042, 3)


,speakerid,phrase,count
0,111113931,0 01,1
1,111113931,0 fervent,1
2,111113931,0 hope,1
3,111113931,0 sugar,1
4,111113931,000 20,1


In [6]:
print(speaker_map.shape)
speaker_map.head()

(112550, 10)


,speakerid,speech_id,lastname,firstname,chamber,state,gender,party,district,nonvoting
0,111120160,1110000007,LARSON,JOHN,H,CT,M,D,1.0,voting
1,111117010,1110000009,PENCE,MIKE,H,IN,M,R,6.0,voting
2,111118060,1110000013,BOEHNER,JOHN,H,OH,M,R,8.0,voting
3,111120780,1110000014,PELOSI,NANCY,H,CA,F,D,8.0,voting
4,111119830,1110000015,HOYER,STENY,H,MD,M,D,5.0,voting


In [7]:
# import speeches from session X
# line 55163 removed extar pipe "vote. (|) 1116 Messrs"
# session_speeches = bigrams = pd.read_csv("gs://rwc1/data/hein-bound/speeches_111.txt", sep = "|", encoding = "ISO-8859-1")

In [8]:
print(session_speeches.shape)
session_speeches.head()

(179268, 2)


,speech_id,speech
0,1110000001,The Representativeselect and their guests will...
1,1110000002,As directed by law. the Clerk of the House has...
2,1110000003,The quor closes that 428 Represer have respond...
3,1110000004,Credentials. regular in form. have been receiv...
4,1110000005,The Clerk is in receipt of a letter of resigna...


In [12]:
print("session_bigrams dim", speaker_bigrams.shape)
print("bigrams_master dim", bigrams_master.shape)

session_bigrams dim (10467042, 3)
bigrams_master dim (6822118, 2)


In [8]:
# generating valid session bigrams
session_bigrams = (speaker_bigrams
                   .merge(bigrams_master, how = "inner", on ="phrase")
                  )

In [9]:
valid_session_bigrams = session_bigrams[session_bigrams['_classify'] == "vocab"]

In [70]:
valid_session_bigrams.head()

,speakerid,phrase,count,_classify
16135,111113931,abil produc,1,vocab
16136,111114091,abil produc,2,vocab
16137,111114101,abil produc,1,vocab
16138,111115360,abil produc,1,vocab
16139,111116310,abil produc,2,vocab


In [12]:
valid_session_bigrams.shape

(5854489, 4)

In [16]:
speaker_groups  = valid_session_bigrams.groupby('speakerid')

In [24]:
for k in speaker_groups.groups.keys():
    print(k)

111113931
111113951
111113981
111114011
111114021
111114091
111114101
111114121
111114171
111114321
111114331
111114451
111114511
111114561
111114621
111114651
111114751
111114941
111115280
111115290
111115300
111115310
111115320
111115330
111115340
111115351
111115360
111115370
111115380
111115390
111115400
111115410
111115420
111115430
111115440
111115450
111115460
111115470
111115480
111115490
111115500
111115510
111115520
111115530
111115540
111115550
111115560
111115570
111115580
111115590
111115600
111115610
111115620
111115630
111115640
111115650
111115660
111115670
111115680
111115690
111115700
111115710
111115720
111115730
111115740
111115750
111115760
111115770
111115780
111115790
111115800
111115810
111115820
111115830
111115840
111115850
111115860
111115870
111115880
111115890
111115900
111115910
111115920
111115930
111115940
111115950
111115960
111115970
111115980
111115990
111116000
111116010
111116020
111116030
111116040
111116050
111116060
111116070
111116080
111116090


In [10]:
# iteration ignoring phrase counts
# speaker_phrases = valid_session_bigrams.groupby('speakerid')['phrase'].apply(lambda x: "[%s]" % ', '.join(x))
speaker_phrases = valid_session_bigrams.groupby('speakerid')['phrase'].apply(list)

In [11]:
speaker_phrases#.apply(len).median()

speakerid
111113931    [abil produc, abil secur, abil strike, abl ass...
111113951    [accomplish thing, accord cbo, across aisl, ac...
111113981    [abl provid, abus practic, accord cbo, account...
111114011    [abl provid, abort right, academ institut, acc...
111114021    [academ standard, accomplish just, achiev awar...
                                   ...                        
111121840    [account offic, across countri, across nation,...
111121930    [abl provid, across countri, agenc congress, a...
111121940    [abund natur, academ institut, accomplish just...
111121950    [across america, across countri, across nation...
111121960    [abl provid, academ success, across countri, a...
Name: phrase, Length: 551, dtype: object

In [16]:
import gensim
dictionary = gensim.corpora.Dictionary(speaker_phrases)

In [17]:
bow_corpus = [dictionary.doc2bow(doc) for doc in speaker_phrases]

In [23]:
len(bow_corpus)

551

In [21]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [24]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=1, workers=2)

In [107]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.000*"need victim" + 0.000*"communiti safeti" + 0.000*"congress commiss" + 0.000*"maintain system" + 0.000*"help survivor" + 0.000*"state dc" + 0.000*"illeg trade" + 0.000*"servic crime" + 0.000*"campus public" + 0.000*"year rebuild"
Topic: 1 
Words: 0.000*"troubl troubl" + 0.000*"head execut" + 0.000*"give thing" + 0.000*"japan korea" + 0.000*"like illeg" + 0.000*"told thing" + 0.000*"spend rememb" + 0.000*"bill hadnt" + 0.000*"elect that" + 0.000*"regul theyr"
Topic: 2 
Words: 0.000*"engin depart" + 0.000*"among strongest" + 0.000*"colleg largest" + 0.000*"servic extraordinari" + 0.000*"sad learn" + 0.000*"becom ever" + 0.000*"firefight everi" + 0.000*"colombia remain" + 0.000*"leftbehind parent" + 0.000*"gave nation"
Topic: 3 
Words: 0.000*"secur imper" + 0.000*"impact unit" + 0.000*"understand occur" + 0.000*"world requir" + 0.000*"stryker vehicl" + 0.000*"author servic" + 0.000*"iran becom" + 0.000*"fuel tank" + 0.000*"support must" + 0.000*"area natur"
Topic: 4 

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [55]:
# speaker with most speeches
speaker_map.groupby('speakerid').agg({'speech_id':'count'}).idxmax()

speech_id    111120961
dtype: int64

In [53]:
# bigram classes
session_bigrams["_classify"].unique()

array(['bad_syntax', 'roberts_and_riddicks', 'co-occurring', 'riddicks',
       'vocab', 'stopword', 'roberts'], dtype=object)

In [38]:
# session speaker count
speaker_map['speakerid'].unique().shape[0]

557

In [46]:
speaker_map.drop(['speech_id'], axis=1).drop_duplicates().head()

,speakerid,lastname,firstname,chamber,state,gender,party,district,nonvoting
0,111120160,LARSON,JOHN,H,CT,M,D,1.0,voting
1,111117010,PENCE,MIKE,H,IN,M,R,6.0,voting
2,111118060,BOEHNER,JOHN,H,OH,M,R,8.0,voting
3,111120780,PELOSI,NANCY,H,CA,F,D,8.0,voting
4,111119830,HOYER,STENY,H,MD,M,D,5.0,voting
